# Visión por Computadora 1
## TP 5 
## Alumno: Santiago Fux

In [147]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt

from datetime import datetime


* Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
* Se deben generar las mascaras de foreground y aplicarlas a los frames
para segmentar los objetos en movimiento.



In [148]:
def calculate_mean(images):
  m = np.mean(images, axis = 0)
  return m.astype('uint8')

In [149]:
def get_random_samples(cap, n):
  #obtengo total de frames
  total = cap.get(cv.CAP_PROP_FRAME_COUNT)
  # verifico max
  if n > total:
    n = total
  # obtengo índices random
  idxs = np.random.randint(total, size=(n))
  return idxs
  


In [150]:
def get_mean_vector(cap, idxs):
  res = []
  # save last position
  curr_frame = cap.get(1) # cv.CV_CAP_PROP_POS_FRAMES
  for i in idxs:
    cap.set(1, i) # cv.CV_CAP_PROP_POS_FRAMES
    ret, frame = cap.read()
    res.append(frame)
  # restore frame
  cap.set(1, curr_frame) # cv.CV_CAP_PROP_POS_FRAMES
  res = np.array(res).astype('uint8')
  return res

In [151]:
def get_ts():
  return np.uint32(datetime.timestamp(datetime.now()))

In [152]:
def naive_bg_sub(cap, n, interval, level):
  # obtengo muestras aleatorias
  exit = False
  next_recalc_ts = 0
  frame_rate = cap.get(5) # cv.CV_CAP_PROP_FPS

  debug_cnt = 0
  while exit == False:
    # obtengo ts actual
    now = get_ts()
    if now > next_recalc_ts:      
      ti = cv.getTickCount()
      # obtengo indices de los frames
      mean_idxs = get_random_samples(cap, n)
      # genero vector con las imagenes(cap) 
      mean_vector = get_mean_vector(cap, mean_idxs)
      # recalculo media de los frames
      mean_img = calculate_mean(mean_vector)
      # Aplicar el filtro de mediana
      mean_img = cv.medianBlur(mean_img, 5)
      # reinicio contador
      next_recalc_ts = now + interval
      tf = cv.getTickCount()
      print(f'recalculando bg...Tiempo={np.uint32((tf-ti) * 1000) / cv.getTickFrequency()}')
    
    #obtengo imagen
    success, img = cap.read()

    if success:
      #guardo tiempo inicial
      ti = cv.getTickCount()
      # hago substraccion de fondo conservando el valor absoluto por si el foreground es más oscuro
      diff_img = cv.absdiff(img, mean_img)
      # Extra: Convierto a escala de grises y luego replico la misma para todos los canales
      diff_img = cv.cvtColor(diff_img, cv.COLOR_BGR2GRAY)
      diff_img = cv.merge([diff_img, diff_img, diff_img])
      # binarizo
      _, diff_img = cv.threshold(diff_img, level, 255,cv.THRESH_BINARY)

      # obtengo las mascaras
      fg_masked = cv.bitwise_and(img, cv.bitwise_not(diff_img))
      bg_masked = cv.bitwise_and(mean_img, diff_img)
      # obtengo la imagen final con sólo el background
      bg_img = cv.add(fg_masked, bg_masked)
      # convierto la imagen a RGB
      bg_img_rgb = cv.cvtColor(bg_img, cv.COLOR_BGR2RGB)      

      # muestro imagen de la máscara
      cv.imshow(f'original', img)
      cv.imshow(f'background only', bg_img)
      cv.imshow(f'mask', diff_img)
      diff_ms = np.uint32((tf-ti) * 1000)/ cv.getTickFrequency()
      print(f'tiempo demandado:{diff_ms}')
    else:
      print(f'Error leyendo video....')
      exit = True

    # verifico si tengo que terminar
    key = cv.waitKey(np.uint32(1000/frame_rate - diff_ms)) & 0xFF
    #cv.destroyAllWindows()
    if key == ord("q"):
      exit = True

  cv.destroyAllWindows()

In [153]:
#procesamos el archivo de gente caminando y pasamos un umbral
n = 10
interval = 30
level = 80
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)
naive_bg_sub(capture, n, interval, level)


recalculando bg...Tiempo=2.947926464
tiempo demandado:1.040853296
tiempo demandado:0.51231088
tiempo demandado:2.37946728
tiempo demandado:2.031117384
tiempo demandado:1.796465488
tiempo demandado:1.306806184
tiempo demandado:1.040597288
tiempo demandado:0.375811392
tiempo demandado:2.700716496
tiempo demandado:2.081516304
tiempo demandado:0.763957112
tiempo demandado:3.569330512
tiempo demandado:2.841081616
tiempo demandado:1.83060972
tiempo demandado:1.561414824
tiempo demandado:0.546656632
tiempo demandado:3.410787736
tiempo demandado:2.362496544
tiempo demandado:1.057737648
tiempo demandado:3.656021752
tiempo demandado:2.88701356
tiempo demandado:2.67137396
tiempo demandado:1.477949768
tiempo demandado:3.961958872
tiempo demandado:2.63425068
tiempo demandado:0.244060488
tiempo demandado:1.608434888
tiempo demandado:4.271649992
tiempo demandado:2.989261504
tiempo demandado:1.225776312
tiempo demandado:1.028309416
tiempo demandado:2.52423952
tiempo demandado:2.843332512
tiempo demand

In [154]:
# procesamos video de autos
n = 10
interval = 30
level = 10
filename = 'slow_traffic_small.mp4'
capture = cv.VideoCapture(filename)
naive_bg_sub(capture, n, interval, level)

recalculando bg...Tiempo=4.244348264
tiempo demandado:0.198319296
tiempo demandado:1.017164736
tiempo demandado:0.865266104
tiempo demandado:2.405557064
tiempo demandado:0.179900432
tiempo demandado:3.558197392
tiempo demandado:3.50646976
tiempo demandado:2.234808832
tiempo demandado:2.2370122
tiempo demandado:2.491096568
tiempo demandado:3.483615936
tiempo demandado:1.583177008
tiempo demandado:3.112696376
tiempo demandado:1.157447448
tiempo demandado:4.120700112
tiempo demandado:2.382870184
tiempo demandado:4.210553552
tiempo demandado:3.187705624
tiempo demandado:0.611824696
tiempo demandado:0.81585136
tiempo demandado:2.146158024
tiempo demandado:0.950393392
tiempo demandado:3.174072056
tiempo demandado:1.842063424
tiempo demandado:3.945520384
tiempo demandado:2.374676456
tiempo demandado:0.828600528
tiempo demandado:2.942634192
tiempo demandado:1.056813264
tiempo demandado:2.140361928
tiempo demandado:2.468442296
tiempo demandado:0.463759368
tiempo demandado:0.374025736
tiempo dem

* Comparar con alguno de los métodos vistos en la practica basados en
mezcla de gaussianas

In [155]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [156]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    

    ti = cv.getTickCount()

    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)

    tf = cv.getTickCount()
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('original', frame)
    cv.imshow('mask', fgMask)

    diff_ms = np.uint32((tf-ti) * 1000)/ cv.getTickFrequency()
    print(f'tiempo demandado:{diff_ms}')
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == ord("q") or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

tiempo demandado:1.656727408
tiempo demandado:3.980594704
tiempo demandado:3.919664704
tiempo demandado:2.774957704
tiempo demandado:3.788410704
tiempo demandado:2.675425704
tiempo demandado:1.465881704
tiempo demandado:3.121746408
tiempo demandado:3.390436704
tiempo demandado:3.444327704
tiempo demandado:0.841046704
tiempo demandado:2.378995704
tiempo demandado:4.216908704
tiempo demandado:3.664121704
tiempo demandado:3.044326704
tiempo demandado:2.797226704
tiempo demandado:0.156570408
tiempo demandado:2.361697704
tiempo demandado:3.499334704
tiempo demandado:1.924847704
tiempo demandado:3.561017704
tiempo demandado:1.259117704
tiempo demandado:4.275162704
tiempo demandado:0.980164704
tiempo demandado:2.827415704
tiempo demandado:3.437108704
tiempo demandado:3.036586704
tiempo demandado:3.899933704
tiempo demandado:2.972915704
tiempo demandado:3.356516704
tiempo demandado:3.257405704
tiempo demandado:2.170141704
tiempo demandado:1.534423704
tiempo demandado:3.083667704
tiempo demanda